In [4]:
import torch
import torch.nn as nn

class VariableLengthLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(VariableLengthLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x should have shape (batch_size, seq_len, input_size)
        out, _ = self.lstm(x)
        print(out.shape)
        # Use the hidden state of the last time step as the output
        out = self.fc(out[:, -1, :])
        return out



In [6]:
# Example usage:
input_size = 10
hidden_size = 20
num_layers = 2
output_size = 1

# Create an instance of the model
model = VariableLengthLSTM(input_size, hidden_size, num_layers, output_size)

# Dummy input data with variable sequence lengths
batch_size = 5
seq_len_1 = 8
seq_len_2 = 10

# Create two sequences with different lengths
input_data_1 = torch.randn(batch_size, seq_len_1, input_size)
input_data_2 = torch.randn(batch_size, seq_len_2, input_size)

# Forward pass
output_1 = model(input_data_1)
print("Output shape for sequence 1:", output_1.shape)

output_2 = model(input_data_2)
print("Output shape for sequence 2:", output_2.shape)



torch.Size([5, 8, 20])
Output shape for sequence 1: torch.Size([5, 1])
torch.Size([5, 10, 20])
Output shape for sequence 2: torch.Size([5, 1])


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GraphConvolutionLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolutionLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adjacency_matrix):
        """
        :param x: Input node features (batch_size, num_nodes, in_features)
        :param adjacency_matrix: Graph adjacency matrix (batch_size, num_nodes, num_nodes)
        :return: Output node features after graph convolution (batch_size, num_nodes, out_features)
        """
        # Assuming the graph is undirected, we use the symmetrically normalized adjacency matrix
        # You may need a different normalization depending on your graph structure
        adjacency_matrix = F.normalize(adjacency_matrix, p=1, dim=2)

        # Perform graph convolution
        x = torch.matmul(adjacency_matrix, x)
        x = self.linear(x)

        return x

class GCN(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super(GCN, self).__init__()
        self.layer1 = GraphConvolutionLayer(input_features, hidden_features)
        self.layer2 = GraphConvolutionLayer(hidden_features, output_features)

    def forward(self, x, adjacency_matrix):
        x = F.relu(self.layer1(x, adjacency_matrix))
        x = self.layer2(x, adjacency_matrix)
        return x



In [8]:
# Example usage:
# Define graph and node features
num_nodes = 10
input_features = 16
hidden_features = 32
output_features = 2

# Create adjacency matrix (assumed undirected and unweighted for simplicity)
adjacency_matrix = torch.randn(1, num_nodes, num_nodes)

# Create input node features
node_features = torch.randn(1, num_nodes, input_features)

# Create GCN model
gcn_model = GCN(input_features, hidden_features, output_features)

# Forward pass
output = gcn_model(node_features, adjacency_matrix)

print("Input node features shape:", node_features.shape)
print("Graph convolution output shape:", output.shape)


Input node features shape: torch.Size([1, 10, 16])
Graph convolution output shape: torch.Size([1, 10, 2])
